This notebook contains code to replicate the workflow and results presented in Section 3.2 of "spectral-recovery: A Python package for spectral recovery analysis of forest ecosystems".

The workflow can be broken down into 4 steps:

1. Read timeseries data/compute indices
2. Define the restoration site
3. Compute a recovery target
4. Compute recovery metrics

In [ ]:
!pip install spectral_recovery

In [ ]:
import spectral_recovery as sr
from spectral_recovery import data

In [ ]:
# 1. Read in the timeseries data and compute indices
ts = sr.read_timeseries(
    path_to_tifs=data.bc06_wildfire_landsat_BAP_timeseries(),
    band_names={},
)
indices = sr.compute_indices(ts, indices=["NBR"])

In [ ]:
# 2. Define the restoration site
#
# Note: the data has disturbance and restoration start dates as attributes on the 
# polygon. We don't need to pass data parameters to the 'read_restoration_polygon' function
# because these attributes are automatically read in.

rest_site = sr.read_restoration_polygons(
    path=data.bc06_wildfire_restoration_site()
)
# TODO: plot the rest site and index data together

In [ ]:
# 3. Compute a historic median recovery target
target = sr.targets.historic.median(
    reference_sites=rest_site,
    timeseries_data=indices,
    reference_years=[2003, 2005]
)

In [ ]:
# 4. Compute R80P and Y2R recovery metrics 
metrics = sr.compute_metrics(
    restoration_polygons=rest_site,
    timeseries_data=indices,
    metrics=["dIR", "R80P", "YrYr", "Y2R"],
    recovery_target=target, 
)

In [ ]:
# TODO: plot metric outputs to show users that the data matches the paper